In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import itertools
from itertools import combinations, chain
import time
import copy
import scipy.stats
from collections import defaultdict
import csv
import sys
import random

In [2]:
# Load all necessary data
ppi = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\PPI data 20_08_2020\\PPI with Uniprot IDs\\PPI.csv', encoding = 'utf-8', compression = 'gzip', low_memory = False, sep = '\t')
uniprot_ids = pd.read_csv(r'D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Network\\Data interaction network\\Metadata\\Filtered metadata\\Metadata.csv', encoding = 'utf-8', compression = 'gzip', low_memory = False, sep = '\t')
ppi2 = ppi.copy()
uniprot_ids = uniprot_ids.astype(str)

# DIAMOnD algorithm

**A DIseAse MOdule Detection (DIAMOnD) Algorithm Derived from a Systematic Analysis of Connectivity Patterns of Disease Proteins in the Human Interactome:**

Disease associated proteins do not reside within locally dense communities and instead identify connectivity significance as the most predictive quantity.
Disease Module Detection (DIAMOnD) algorithm to identify the full disease module around a set of known disease proteins.

Building on the observation that the connectivity significance is highly distinctive for known disease proteins, we propose the following algorithm to infer yet unknown disease proteins, and hence to identify the respective disease module:

* The connectivity significance is determined for all proteins connected to any of the s0 seed proteins.
* The proteins are ranked according to their respective p-values.
* The protein with the highest rank (i.e. lowest p-value) is added to the set of seed nodes, increasing their number from s0 →s1 = s0+1.
* Steps (i)-(iii) are repeated with the expanded set of seed proteins, pulling in one protein at a time into the growing disease module.

In [3]:
# https://github.com/dinaghiassian/DIAMOnD

# =============================================================================
print(' ')
print('        usage: python3 DIAMOnD.py network_file seed_file n alpha(optional) outfile_name (optional)')
print('        -----------------------------------------------------------------')
print('        network_file : The edgelist must be provided as any delimiter-separated')
print('                       table. Make sure the delimiter does not exit in gene IDs')
print('                       and is consistent across the file.')
print('                       The first two columns of the table will be')
print('                       interpreted as an interaction gene1 <==> gene2')
print('        seed_file    : table containing the seed genes (if table contains')
print('                       more than one column they must be tab-separated;')
print('                       the first column will be used only)')
print('        n            : desired number of DIAMOnD genes, 200 is a reasonable')
print('                       starting point.')
print('        alpha        : an integer representing weight of the seeds,default')
print('                       value is set to 1')
print('        outfile_name : results will be saved under this file name')
print('                       by default the outfile_name is set to "first_n_added_nodes_weight_alpha.txt"')
print(' ')


# =============================================================================

def read_input(network_file, seed_file):
    """
    Reads the network and the list of seed genes from external files.
    * The edgelist must be provided as a tab-separated table. The
    first two columns of the table will be interpreted as an
    interaction gene1 <==> gene2
    * The seed genes mus be provided as a table. If the table has more
    than one column, they must be tab-separated. The first column will
    be used only.
    * Lines that start with '#' will be ignored in both cases
    """

    sniffer = csv.Sniffer()
    line_delimiter = None
    for line in open(network_file, 'r'):
        if line[0] == '#':
            continue
        else:
            dialect = sniffer.sniff(line)
            line_delimiter = dialect.delimiter
            break
    if line_delimiter == None:
        print
        'network_file format not correct'
        sys.exit(0)

    # read the network:
    G = nx.Graph()
    for line in open(network_file, 'r'):
        # lines starting with '#' will be ignored
        if line[0] == '#':
            continue
        # The first two columns in the line will be interpreted as an
        # interaction gene1 <=> gene2
        # line_data   = line.strip().split('\t')
        line_data = line.strip().split(line_delimiter)
        node1 = line_data[0]
        node2 = line_data[1]
        G.add_edge(node1, node2)

    # read the seed genes:
    seed_genes = set()
    for line in open(seed_file, 'r'):
        # lines starting with '#' will be ignored
        if line[0] == '#':
            continue
        # the first column in the line will be interpreted as a seed
        # gene:
        line_data = line.strip().split('\t')
        seed_gene = line_data[0]
        seed_genes.add(seed_gene)

    return G, seed_genes


# ================================================================================
def compute_all_gamma_ln(N):
    """
    precomputes all logarithmic gammas
    """
    gamma_ln = {}
    for i in range(1, N + 1):
        gamma_ln[i] = scipy.special.gammaln(i)

    return gamma_ln


# =============================================================================
def logchoose(n, k, gamma_ln):
    if n - k + 1 <= 0:
        return scipy.infty
    lgn1 = gamma_ln[n + 1]
    lgk1 = gamma_ln[k + 1]
    lgnk1 = gamma_ln[n - k + 1]
    return lgn1 - [lgnk1 + lgk1]


# =============================================================================
def gauss_hypergeom(x, r, b, n, gamma_ln):
    return np.exp(logchoose(r, x, gamma_ln) +
                  logchoose(b, n - x, gamma_ln) -
                  logchoose(r + b, n, gamma_ln))


# =============================================================================
def pvalue(kb, k, N, s, gamma_ln):
    """
    -------------------------------------------------------------------
    Computes the p-value for a node that has kb out of k links to
    seeds, given that there's a total of s sees in a network of N nodes.
    p-val = \sum_{n=kb}^{k} HypergemetricPDF(n,k,N,s)
    -------------------------------------------------------------------
    """
    p = 0.0
    for n in range(kb, k + 1):
        if n > s:
            break
        prob = gauss_hypergeom(n, s, N - s, k, gamma_ln)
        # print prob
        p += prob

    if p > 1:
        return 1
    else:
        return p

    # =============================================================================


def get_neighbors_and_degrees(G):
    neighbors, all_degrees = {}, {}
    for node in G.nodes():
        nn = set(G.neighbors(node))
        neighbors[node] = nn
        all_degrees[node] = G.degree(node)

    return neighbors, all_degrees


# =============================================================================
# Reduce number of calculations
# =============================================================================
def reduce_not_in_cluster_nodes(all_degrees, neighbors, G, not_in_cluster, cluster_nodes, alpha):
    reduced_not_in_cluster = {}
    kb2k = defaultdict(dict)
    for node in not_in_cluster:

        k = all_degrees[node]
        kb = 0
        # Going through all neighbors and counting the number of module neighbors
        for neighbor in neighbors[node]:
            if neighbor in cluster_nodes:
                kb += 1

        # adding wights to the the edges connected to seeds
        k += (alpha - 1) * kb
        kb += (alpha - 1) * kb
        kb2k[kb][k] = node

    # Going to choose the node with largest kb, given k
    k2kb = defaultdict(dict)
    for kb, k2node in kb2k.items():
        min_k = min(k2node.keys())
        node = k2node[min_k]
        k2kb[min_k][kb] = node

    for k, kb2node in k2kb.items():
        max_kb = max(kb2node.keys())
        node = kb2node[max_kb]
        reduced_not_in_cluster[node] = (max_kb, k)

    return reduced_not_in_cluster


# ======================================================================================
#   C O R E    A L G O R I T H M
# ======================================================================================
def diamond_iteration_of_first_X_nodes(G, S, X, alpha):
    """
    Parameters:
    ----------
    - G:     graph
    - S:     seeds
    - X:     the number of iterations, i.e only the first X gened will be
             pulled in
    - alpha: seeds weight
    Returns:
    --------
    - added_nodes: ordered list of nodes in the order by which they
      are agglomerated. Each entry has 4 info:
      * name : dito
      * k    : degree of the node
      * kb   : number of +1 neighbors
      * p    : p-value at agglomeration
    """

    N = G.number_of_nodes()

    added_nodes = []

    # ------------------------------------------------------------------
    # Setting up dictionaries with all neighbor lists
    # and all degrees
    # ------------------------------------------------------------------
    neighbors, all_degrees = get_neighbors_and_degrees(G)

    # ------------------------------------------------------------------
    # Setting up initial set of nodes in cluster
    # ------------------------------------------------------------------

    cluster_nodes = set(S)
    not_in_cluster = set()
    s0 = len(cluster_nodes)

    s0 += (alpha - 1) * s0
    N += (alpha - 1) * s0

    # ------------------------------------------------------------------
    # precompute the logarithmic gamma functions
    # ------------------------------------------------------------------
    gamma_ln = compute_all_gamma_ln(N + 1)

    # ------------------------------------------------------------------
    # Setting initial set of nodes not in cluster
    # ------------------------------------------------------------------
    for node in cluster_nodes:
        not_in_cluster |= neighbors[node]
    not_in_cluster -= cluster_nodes

    # ------------------------------------------------------------------
    #
    # M A I N     L O O P
    #
    # ------------------------------------------------------------------

    all_p = {}

    while len(added_nodes) < X:

        # ------------------------------------------------------------------
        #
        # Going through all nodes that are not in the cluster yet and
        # record k, kb and p
        #
        # ------------------------------------------------------------------

        info = {}

        pmin = 10
        next_node = 'nix'
        reduced_not_in_cluster = reduce_not_in_cluster_nodes(all_degrees,
                                                             neighbors, G,
                                                             not_in_cluster,
                                                             cluster_nodes, alpha)

        for node, kbk in reduced_not_in_cluster.items():
            # Getting the p-value of this kb,k
            # combination and save it in all_p, so computing it only once!
            kb, k = kbk
            try:
                p = all_p[(k, kb, s0)]
            except KeyError:
                p = pvalue(kb, k, N, s0, gamma_ln)
                all_p[(k, kb, s0)] = p

            # recording the node with smallest p-value
            if p < pmin:
                pmin = p
                next_node = node

            info[node] = (k, kb, p)

        # ---------------------------------------------------------------------
        # Adding node with smallest p-value to the list of aaglomerated nodes
        # ---------------------------------------------------------------------
        added_nodes.append((next_node,
                            info[next_node][0],
                            info[next_node][1],
                            info[next_node][2]))

        # Updating the list of cluster nodes and s0
        cluster_nodes.add(next_node)
        s0 = len(cluster_nodes)
        not_in_cluster |= (neighbors[next_node] - cluster_nodes)
        not_in_cluster.remove(next_node)

    return added_nodes


# ===========================================================================
#
#   M A I N    D I A M O n D    A L G O R I T H M
#
# ===========================================================================
def DIAMOnD(G_original, seed_genes, max_number_of_added_nodes, alpha, outfile=None):
    """
    Runs the DIAMOnD algorithm
    Input:
    ------
     - G_original :
             The network
     - seed_genes :
             a set of seed genes
     - max_number_of_added_nodes:
             after how many added nodes should the algorithm stop
     - alpha:
             given weight to the sees
     - outfile:
             filename for the output generates by the algorithm,
             if not given the program will name it 'first_x_added_nodes.txt'
     Returns:
     --------
      - added_nodes: A list with 4 entries at each element:
            * name : name of the node
            * k    : degree of the node
            * kb   : number of neighbors that are part of the module (at agglomeration)
            * p    : connectivity p-value at agglomeration
      -
    """

    # 1. throwing away the seed genes that are not in the network
    all_genes_in_network = set(G_original.nodes())
    seed_genes = set(seed_genes)
    disease_genes = seed_genes & all_genes_in_network

    if len(disease_genes) != len(seed_genes):
        print("DIAMOnD(): ignoring %s of %s seed genes that are not in the network" % (
            len(seed_genes - all_genes_in_network), len(seed_genes)))

    # 2. agglomeration algorithm.
    added_nodes = diamond_iteration_of_first_X_nodes(G_original,
                                                     disease_genes,
                                                     max_number_of_added_nodes, alpha)
    # 3. saving the results
    with open(outfile, 'w') as fout:
        fout.write('\t'.join(['#rank', 'DIAMOnD_node', 'p_hyper']) + '\n')
        rank = 0
        for DIAMOnD_node_info in added_nodes:
            rank += 1
            DIAMOnD_node = DIAMOnD_node_info[0]
            p = float(DIAMOnD_node_info[3])

            fout.write('\t'.join(map(str, ([rank, DIAMOnD_node, p]))) + '\n')
    return added_nodes

 
        usage: python3 DIAMOnD.py network_file seed_file n alpha(optional) outfile_name (optional)
        -----------------------------------------------------------------
        network_file : The edgelist must be provided as any delimiter-separated
                       table. Make sure the delimiter does not exit in gene IDs
                       and is consistent across the file.
                       The first two columns of the table will be
                       interpreted as an interaction gene1 <==> gene2
        seed_file    : table containing the seed genes (if table contains
                       more than one column they must be tab-separated;
                       the first column will be used only)
        n            : desired number of DIAMOnD genes, 200 is a reasonable
                       starting point.
        alpha        : an integer representing weight of the seeds,default
                       value is set to 1
        outfile_name : results will

# Check the most porminent diseases in the protein-protein interaction network

In [4]:
# Get all the uniprot IDs in the ass network and check which ones are present in the uniprot
ppi_uniprot_ids = list(set(list(ppi2.UniprotAccession_A) + list(ppi2.UniprotAccession_B)))
uniprot_ppi = uniprot_ids[uniprot_ids['UniprotAccession'].isin(ppi_uniprot_ids)]
uniprot_ppi = uniprot_ppi.astype(str)

# Create a new dataframe containing the disease terms and the amount they are present in the original dataframe
disease_counts_ppi = pd.DataFrame(uniprot_ppi['DisGeNet_disease_name'].str.split('; ', expand=True).stack().value_counts()).reset_index()
disease_counts_ppi.columns = ['Diseases', 'Count']
disease_counts_ppi = disease_counts_ppi.drop([0]).reset_index(drop = True)

#display the top 30 disease terms and the amount of times they are present7
disease_counts_ppi.head(10)

,Diseases,Count
0,Schizophrenia,365
1,Malignant neoplasm of breast,359
2,Breast Carcinoma,307
3,Malignant neoplasm of prostate,225
4,Intellectual Disability,210
5,Liver carcinoma,203
6,Colorectal Carcinoma,180
7,Mammary Neoplasms,153
8,Hypertensive disease,137
9,Obesity,135


# Optimize the parameters

## Check how many disease-related proteins are in the network and create a randomized sample op nodes to optimize the parameters

In [5]:
# Identify all proteins that are associated with Breast Carcinoma in the network and 
# filter only the nodes that have annotations
disease_ppi = uniprot_ppi[uniprot_ppi['DisGeNet_disease_name'].str.contains('Breast Carcinoma')].reset_index(drop = True)

x = ['GO_biological_process', 
     'GO_cellular_component', 
     'GO_molecular_function', 
     'Reactome_ID', 
     'HPA_Subcellular_location']
for ele in x:
    disease_ppi = disease_ppi[disease_ppi[ele] != 'nan']

seeds = disease_ppi.UniprotAccession.tolist()

# Create all empty dataframes
x = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

# No filtration
df_5 = pd.DataFrame()
df_5['Amount of added proteins'], \
df_5['alpha = 1'], \
df_5['alpha = 2'], \
df_5['alpha = 3'], \
df_5['alpha = 4'], \
df_5['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                    [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_5 = df_5.set_index('Amount of added proteins')

df_10 = pd.DataFrame()
df_10['Amount of added proteins'], \
df_10['alpha = 1'], \
df_10['alpha = 2'], \
df_10['alpha = 3'], \
df_10['alpha = 4'], \
df_10['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                     [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_10 = df_10.set_index('Amount of added proteins')

df_15 = pd.DataFrame()
df_15['Amount of added proteins'], \
df_15['alpha = 1'], \
df_15['alpha = 2'], \
df_15['alpha = 3'], \
df_15['alpha = 4'], \
df_15['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                     [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_15 = df_15.set_index('Amount of added proteins')

# Reactome general filtration
df_p5 = pd.DataFrame()
df_p5['Amount of added proteins'], \
df_p5['alpha = 1'], \
df_p5['alpha = 2'], \
df_p5['alpha = 3'], \
df_p5['alpha = 4'], \
df_p5['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                     [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_p5 = df_p5.set_index('Amount of added proteins')

df_p10 = pd.DataFrame()
df_p10['Amount of added proteins'], \
df_p10['alpha = 1'], \
df_p10['alpha = 2'], \
df_p10['alpha = 3'], \
df_p10['alpha = 4'], \
df_p10['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                      [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_p10 = df_p10.set_index('Amount of added proteins')

df_p15 = pd.DataFrame()
df_p15['Amount of added proteins'], \
df_p15['alpha = 1'], \
df_p15['alpha = 2'], \
df_p15['alpha = 3'], \
df_p15['alpha = 4'], \
df_p15['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                      [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_p15 = df_p15.set_index('Amount of added proteins')

# Reactome leaf pathway filtration
df_lp5 = pd.DataFrame()
df_lp5['Amount of added proteins'], \
df_lp5['alpha = 1'], \
df_lp5['alpha = 2'], \
df_lp5['alpha = 3'], \
df_lp5['alpha = 4'], \
df_lp5['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                      [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_lp5 = df_lp5.set_index('Amount of added proteins')

df_lp10 = pd.DataFrame()
df_lp10['Amount of added proteins'], \
df_lp10['alpha = 1'], \
df_lp10['alpha = 2'], \
df_lp10['alpha = 3'], \
df_lp10['alpha = 4'], \
df_lp10['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                       [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_lp10 = df_lp10.set_index('Amount of added proteins')

df_lp15 = pd.DataFrame()
df_lp15['Amount of added proteins'], \
df_lp15['alpha = 1'], \
df_lp15['alpha = 2'], \
df_lp15['alpha = 3'], \
df_lp15['alpha = 4'], \
df_lp15['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                       [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_lp15 = df_lp15.set_index('Amount of added proteins')

# Subcellular location filtration
df_l5 = pd.DataFrame()
df_l5['Amount of added proteins'], \
df_l5['alpha = 1'], \
df_l5['alpha = 2'], \
df_l5['alpha = 3'], \
df_l5['alpha = 4'], \
df_l5['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                     [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_l5 = df_l5.set_index('Amount of added proteins')

df_l10 = pd.DataFrame()
df_l10['Amount of added proteins'], \
df_l10['alpha = 1'], \
df_l10['alpha = 2'], \
df_l10['alpha = 3'], \
df_l10['alpha = 4'], \
df_l10['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                      [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_l10 = df_l10.set_index('Amount of added proteins')

df_l15 = pd.DataFrame()
df_l15['Amount of added proteins'], \
df_l15['alpha = 1'], \
df_l15['alpha = 2'], \
df_l15['alpha = 3'], \
df_l15['alpha = 4'], \
df_l15['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                      [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_l15 = df_l15.set_index('Amount of added proteins')

# GO filtration
df_go5 = pd.DataFrame()
df_go5['Amount of added proteins'], \
df_go5['alpha = 1'], \
df_go5['alpha = 2'], \
df_go5['alpha = 3'], \
df_go5['alpha = 4'], \
df_go5['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                      [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_go5 = df_go5.set_index('Amount of added proteins')

df_go10 = pd.DataFrame()
df_go10['Amount of added proteins'], \
df_go10['alpha = 1'], \
df_go10['alpha = 2'], \
df_go10['alpha = 3'], \
df_go10['alpha = 4'], \
df_go10['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                       [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_go10 = df_go10.set_index('Amount of added proteins')

df_go15 = pd.DataFrame()
df_go15['Amount of added proteins'], \
df_go15['alpha = 1'], \
df_go15['alpha = 2'], \
df_go15['alpha = 3'], \
df_go15['alpha = 4'], \
df_go15['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                       [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_go15 = df_go15.set_index('Amount of added proteins')

# General reactome filtration + subcellular location filtration
df_pl5 = pd.DataFrame()
df_pl5['Amount of added proteins'], \
df_pl5['alpha = 1'], \
df_pl5['alpha = 2'], \
df_pl5['alpha = 3'], \
df_pl5['alpha = 4'], \
df_pl5['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                      [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_pl5 = df_pl5.set_index('Amount of added proteins')

df_pl10 = pd.DataFrame()
df_pl10['Amount of added proteins'], \
df_pl10['alpha = 1'], \
df_pl10['alpha = 2'], \
df_pl10['alpha = 3'], \
df_pl10['alpha = 4'], \
df_pl10['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                       [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_pl10 = df_pl10.set_index('Amount of added proteins')

df_pl15 = pd.DataFrame()
df_pl15['Amount of added proteins'], \
df_pl15['alpha = 1'], \
df_pl15['alpha = 2'], \
df_pl15['alpha = 3'], \
df_pl15['alpha = 4'], \
df_pl15['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                       [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_pl15 = df_pl15.set_index('Amount of added proteins')

# GO, Reactome and Subcellular location filtering
df_gopl5 = pd.DataFrame()
df_gopl5['Amount of added proteins'], \
df_gopl5['alpha = 1'], \
df_gopl5['alpha = 2'], \
df_gopl5['alpha = 3'], \
df_gopl5['alpha = 4'], \
df_gopl5['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                        [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_gopl5 = df_gopl5.set_index('Amount of added proteins')

df_gopl10 = pd.DataFrame()
df_gopl10['Amount of added proteins'], \
df_gopl10['alpha = 1'], \
df_gopl10['alpha = 2'], \
df_gopl10['alpha = 3'], \
df_gopl10['alpha = 4'], \
df_gopl10['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                         [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_gopl10 = df_gopl10.set_index('Amount of added proteins')

df_gopl15 = pd.DataFrame()
df_gopl15['Amount of added proteins'], \
df_gopl15['alpha = 1'], \
df_gopl15['alpha = 2'], \
df_gopl15['alpha = 3'], \
df_gopl15['alpha = 4'], \
df_gopl15['alpha = 5'] = [ele for ele in x], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], \
                         [0.0 for i in range(len(x))], [0.0 for i in range(len(x))], [0.0 for i in range(len(x))]
df_gopl15 = df_gopl15.set_index('Amount of added proteins')

# Create the different networks and filtration steps already
# No filtration
nodes = list(set(list(ppi2.UniprotAccession_A.unique()) + list(ppi2.UniprotAccession_B.unique())))
edges = []
for i in range(len(ppi2)):
    edges.append(tuple([ppi2.UniprotAccession_A[i], ppi2.UniprotAccession_B[i]]))

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

# Reactome general filtration
uniprot_p = uniprot_ppi.astype(str).reset_index(drop = True)
disease_ppi = uniprot_ppi[uniprot_ppi['DisGeNet_disease_name'].str.contains('Breast Carcinoma')].reset_index(drop = True)
disease_ppi = disease_ppi[disease_ppi.UniprotAccession.isin(seeds)].reset_index(drop = True)
disease_filtered = uniprot_p.copy().reset_index(drop = True)
pathways_disease = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.Reactome_ID[i]).split('; ')
    for e in range(len(x)):
        pathways_disease.add(x[e])
pathways_disease = list(pathways_disease)
if 'nan' in pathways_disease:
    pathways_disease.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(pathways_disease).intersection(disease_filtered.Reactome_ID[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
nodes_disease = disease_filtered.UniprotAccession
edges_disease_ppi = pd.DataFrame(ppi2[ppi2['UniprotAccession_A'].isin(nodes_disease) & \
                                      ppi2['UniprotAccession_B'].isin(nodes_disease)])
edges_disease_ppi.reset_index(drop=True, inplace = True)
edges_disease = []
for i in range(len(edges_disease_ppi)):
    edges_disease.append(tuple([edges_disease_ppi.UniprotAccession_A[i], edges_disease_ppi.UniprotAccession_B[i]]))

G_p = nx.Graph()
G_p.add_nodes_from(nodes_disease)
G_p.add_edges_from(edges_disease)

# Reactome leaf pathway filtration
reactome_leaf = pd.read_table("https://reactome.org/download/current/UniProt2Reactome.txt", 
                                   sep = '\t', 
                                   names = ['UniprotAccession',
                                            'Reactome_ID', 
                                            'URL', 
                                            'Reactome_name', 
                                            'Evidence Code', 
                                            'Species'], index_col=False)
reactome_leaf = reactome_leaf.astype(str)
reactome_leaf = reactome_leaf[reactome_leaf['Species'] == 'Homo sapiens'].reset_index(drop = True)
reactome_leaf = reactome_leaf[['UniprotAccession', 'Reactome_ID']]
uniprot_lp = uniprot_ppi.astype(str).reset_index(drop = True)
disease_ppi = uniprot_ppi[uniprot_ppi['DisGeNet_disease_name'].str.contains('Breast Carcinoma')].reset_index(drop = True)
disease_ppi = disease_ppi[disease_ppi.UniprotAccession.isin(seeds)].reset_index(drop = True)
disease_filtered = uniprot_lp.copy().reset_index(drop = True)
reactome_leaf = reactome_leaf[reactome_leaf['UniprotAccession'].isin(seeds)].drop_duplicates().reset_index(drop = True)
leaf_pathways_disease = reactome_leaf.Reactome_ID.unique().tolist()
for i in range(disease_filtered.shape[0]):
    if len(set(leaf_pathways_disease).intersection(disease_filtered.Reactome_ID[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
nodes_disease = disease_filtered.UniprotAccession
edges_disease_ppi = pd.DataFrame(ppi2[ppi2['UniprotAccession_A'].isin(nodes_disease) & \
                                      ppi2['UniprotAccession_B'].isin(nodes_disease)])
edges_disease_ppi.reset_index(drop=True, inplace = True)
edges_disease = []
for i in range(len(edges_disease_ppi)):
    edges_disease.append(tuple([edges_disease_ppi.UniprotAccession_A[i], edges_disease_ppi.UniprotAccession_B[i]]))

G_lp = nx.Graph()
G_lp.add_nodes_from(nodes_disease)
G_lp.add_edges_from(edges_disease)

# Subcellular Location filtration
reactome_leaf = pd.read_table("https://reactome.org/download/current/UniProt2Reactome.txt", 
                                   sep = '\t', 
                                   names = ['UniprotAccession',
                                            'Reactome_ID', 
                                            'URL', 
                                            'Reactome_name', 
                                            'Evidence Code', 
                                            'Species'], index_col=False)
reactome_leaf = reactome_leaf.astype(str)
reactome_leaf = reactome_leaf[reactome_leaf['Species'] == 'Homo sapiens'].reset_index(drop = True)
reactome_leaf = reactome_leaf[['UniprotAccession', 'Reactome_ID']]
uniprot_lp = uniprot_ppi.astype(str).reset_index(drop = True)
disease_ppi = uniprot_ppi[uniprot_ppi['DisGeNet_disease_name'].str.contains('Breast Carcinoma')].reset_index(drop = True)
disease_ppi = disease_ppi[disease_ppi.UniprotAccession.isin(seeds)].reset_index(drop = True)
disease_filtered = uniprot_lp.copy().reset_index(drop = True)
reactome_leaf = reactome_leaf[reactome_leaf['UniprotAccession'].isin(seeds)].drop_duplicates().reset_index(drop = True)
leaf_pathways_disease = reactome_leaf.Reactome_ID.unique().tolist()
for i in range(disease_filtered.shape[0]):
    if len(set(leaf_pathways_disease).intersection(disease_filtered.Reactome_ID[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
nodes_disease = disease_filtered.UniprotAccession
edges_disease_ppi = pd.DataFrame(ppi2[ppi2['UniprotAccession_A'].isin(nodes_disease) & \
                                      ppi2['UniprotAccession_B'].isin(nodes_disease)])
edges_disease_ppi.reset_index(drop=True, inplace = True)
edges_disease = []
for i in range(len(edges_disease_ppi)):
    edges_disease.append(tuple([edges_disease_ppi.UniprotAccession_A[i], edges_disease_ppi.UniprotAccession_B[i]]))

G_l = nx.Graph()
G_l.add_nodes_from(nodes_disease)
G_l.add_edges_from(edges_disease)

# GO filtration
uniprot_go = uniprot_ppi.astype(str).reset_index(drop = True)
disease_ppi = uniprot_ppi[uniprot_ppi['DisGeNet_disease_name'].str.contains('Breast Carcinoma')].reset_index(drop = True)
disease_ppi = disease_ppi[disease_ppi.UniprotAccession.isin(seeds)].reset_index(drop = True)
disease_filtered = uniprot_go.copy().reset_index(drop = True)
GObp = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.GO_biological_process[i]).split('; ')
    for e in range(len(x)):
        GObp.add(x[e])
GObp = list(GObp)
if 'nan' in GObp:
    GObp.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(GObp).intersection(disease_filtered.GO_biological_process[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
GOmf = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.GO_molecular_function[i]).split('; ')
    for e in range(len(x)):
        GOmf.add(x[e])
GOmf = list(GOmf)
if 'nan' in GOmf:
    GOmf.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(GOmf).intersection(disease_filtered.GO_molecular_function[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
GOcc = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.GO_cellular_component[i]).split('; ')
    for e in range(len(x)):
        GOcc.add(x[e])
GOcc = list(GOcc)
if 'nan' in GOcc:
    GOcc.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(GOcc).intersection(disease_filtered.GO_cellular_component[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
# Select the protein names and retain only the edges betweeen those nodes
nodes_disease = disease_filtered.UniprotAccession
edges_disease_ppi = pd.DataFrame(ppi2[ppi2['UniprotAccession_A'].isin(nodes_disease) & \
                                      ppi2['UniprotAccession_B'].isin(nodes_disease)])
edges_disease_ppi.reset_index(drop=True, inplace = True)
edges_disease = []
for i in range(len(edges_disease_ppi)):
    edges_disease.append(tuple([edges_disease_ppi.UniprotAccession_A[i], edges_disease_ppi.UniprotAccession_B[i]]))

G_go = nx.Graph()
G_go.add_nodes_from(nodes_disease)
G_go.add_edges_from(edges_disease)

# General reactome filtration + subcellular location filtration
uniprot_pl = uniprot_ppi.astype(str).reset_index(drop = True)
disease_ppi = uniprot_ppi[uniprot_ppi['DisGeNet_disease_name'].str.contains('Breast Carcinoma')].reset_index(drop = True)
disease_ppi = disease_ppi[disease_ppi.UniprotAccession.isin(seeds)].reset_index(drop = True)
disease_filtered = uniprot_pl.copy().reset_index(drop = True)
pathways_disease = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.Reactome_ID[i]).split('; ')
    for e in range(len(x)):
        pathways_disease.add(x[e])
pathways_disease = list(pathways_disease)
if 'nan' in pathways_disease:
    pathways_disease.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(pathways_disease).intersection(disease_filtered.Reactome_ID[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
location_disease = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.HPA_Subcellular_location[i]).split(',')
    for e in range(len(x)):
        location_disease.add(x[e])
location_disease = list(location_disease)
if 'nan' in location_disease:
    location_disease.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(set(location_disease).intersection(disease_filtered.HPA_Subcellular_location[i].split(',')))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
nodes_disease = disease_filtered.UniprotAccession
edges_disease_ppi = pd.DataFrame(ppi2[ppi2['UniprotAccession_A'].isin(nodes_disease) & \
                                      ppi2['UniprotAccession_B'].isin(nodes_disease)])
edges_disease_ppi.reset_index(drop=True, inplace = True)
edges_disease = []
for i in range(len(edges_disease_ppi)):
    edges_disease.append(tuple([edges_disease_ppi.UniprotAccession_A[i], edges_disease_ppi.UniprotAccession_B[i]]))

G_pl = nx.Graph()
G_pl.add_nodes_from(nodes_disease)
G_pl.add_edges_from(edges_disease)

# GO, Reactome and Subcellular location filtering
uniprot_gopl = uniprot_ppi.astype(str).reset_index(drop = True)
disease_ppi = uniprot_ppi[uniprot_ppi['DisGeNet_disease_name'].str.contains('Breast Carcinoma')].reset_index(drop = True)
disease_ppi = disease_ppi[disease_ppi.UniprotAccession.isin(seeds)].reset_index(drop = True)
disease_filtered = uniprot_gopl.copy().reset_index(drop = True)
GObp = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.GO_biological_process[i]).split('; ')
    for e in range(len(x)):
        GObp.add(x[e])
GObp = list(GObp)
if 'nan' in GObp:
    GObp.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(GObp).intersection(disease_filtered.GO_biological_process[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
GOmf = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.GO_molecular_function[i]).split('; ')
    for e in range(len(x)):
        GOmf.add(x[e])
GOmf = list(GOmf)
if 'nan' in GOmf:
    GOmf.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(GOmf).intersection(disease_filtered.GO_molecular_function[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
GOcc = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.GO_cellular_component[i]).split('; ')
    for e in range(len(x)):
        GOcc.add(x[e])
GOcc = list(GOcc)
if 'nan' in GOcc:
    GOcc.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(GOcc).intersection(disease_filtered.GO_cellular_component[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
pathways_disease = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.Reactome_ID[i]).split('; ')
    for e in range(len(x)):
        pathways_disease.add(x[e])
pathways_disease = list(pathways_disease)
if 'nan' in pathways_disease:
    pathways_disease.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(pathways_disease).intersection(disease_filtered.Reactome_ID[i].split('; '))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
location_disease = set()
for i in range(len(disease_ppi)):
    x = str(disease_ppi.HPA_Subcellular_location[i]).split(',')
    for e in range(len(x)):
        location_disease.add(x[e])
location_disease = list(location_disease)
if 'nan' in location_disease:
    location_disease.remove('nan')
for i in range(disease_filtered.shape[0]):
    if len(set(set(location_disease).intersection(disease_filtered.HPA_Subcellular_location[i].split(',')))) == 0:
        disease_filtered.drop([i], inplace = True)
disease_filtered.reset_index(drop=True, inplace = True)
nodes_disease = disease_filtered.UniprotAccession
edges_disease_ppi = pd.DataFrame(ppi2[ppi2['UniprotAccession_A'].isin(nodes_disease) & \
                                      ppi2['UniprotAccession_B'].isin(nodes_disease)])
edges_disease_ppi.reset_index(drop=True, inplace = True)
edges_disease = []
for i in range(len(edges_disease_ppi)):
    edges_disease.append(tuple([edges_disease_ppi.UniprotAccession_A[i], edges_disease_ppi.UniprotAccession_B[i]]))

G_gopl = nx.Graph()
G_gopl.add_nodes_from(nodes_disease)
G_gopl.add_edges_from(edges_disease)

In [6]:
# Repeat all steps 20 times and get an average of the 20 times by the end
for _ in range(10):
    seeds1 = random.sample(seeds, len(seeds)-5)
    seeds2 = random.sample(seeds1, len(seeds)-10)
    seeds3 = random.sample(seeds2, len(seeds)-15)
    
    x = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
    
    # No Filtration
    for ele in x:
        for i in range(1, 6):
            added_nodes5 = set()
            added_nodes10 = set()
            added_nodes15 = set()
            algo5 = DIAMOnD(G_original = G, seed_genes = seeds1, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random5.csv')
            algo10 = DIAMOnD(G_original = G, seed_genes = seeds2, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random10.csv')
            algo15 = DIAMOnD(G_original = G, seed_genes = seeds3, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random15.csv')
            for e in range(ele):
                added_nodes5.add(algo5[e][0])
                added_nodes10.add(algo10[e][0])
                added_nodes15.add(algo15[e][0])
            seeds_intersect5 = set(added_nodes5.intersection(seeds))
            seeds_intersect10 = set(added_nodes10.intersection(seeds))
            seeds_intersect15 = set(added_nodes15.intersection(seeds))
            df_5[f'alpha = {i}'][ele] += round(len(seeds_intersect5)/5 * 100, 2)
            df_10[f'alpha = {i}'][ele] += round(len(seeds_intersect10)/10 * 100, 2)
            df_15[f'alpha = {i}'][ele] += round(len(seeds_intersect15)/15 * 100, 2)
    
    # Reactome general filtration
    for ele in x:
        for i in range(1, 6):
            added_nodes5 = set()
            added_nodes10 = set()
            added_nodes15 = set()
            algo5 = DIAMOnD(G_original = G_p, seed_genes = seeds1, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random5.csv')
            algo10 = DIAMOnD(G_original = G_p, seed_genes = seeds2, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random10.csv')
            algo15 = DIAMOnD(G_original = G_p, seed_genes = seeds3, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random15.csv')
            for e in range(ele):
                added_nodes5.add(algo5[e][0])
                added_nodes10.add(algo10[e][0])
                added_nodes15.add(algo15[e][0])
            seeds_intersect5 = set(added_nodes5.intersection(seeds))
            seeds_intersect10 = set(added_nodes10.intersection(seeds))
            seeds_intersect15 = set(added_nodes15.intersection(seeds))
            df_p5[f'alpha = {i}'][ele] += round(len(seeds_intersect5)/5 * 100, 2)
            df_p10[f'alpha = {i}'][ele] += round(len(seeds_intersect10)/10 * 100, 2)
            df_p15[f'alpha = {i}'][ele] += round(len(seeds_intersect15)/15 * 100, 2)
    
    # Reactome leaf pathway filtration
    for ele in x:
        for i in range(1, 6):
            added_nodes5 = set()
            added_nodes10 = set()
            added_nodes15 = set()
            algo5 = DIAMOnD(G_original = G_lp, seed_genes = seeds1, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random5.csv')
            algo10 = DIAMOnD(G_original = G_lp, seed_genes = seeds2, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random10.csv')
            algo15 = DIAMOnD(G_original = G_lp, seed_genes = seeds3, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random15.csv')
            for e in range(ele):
                added_nodes5.add(algo5[e][0])
                added_nodes10.add(algo10[e][0])
                added_nodes15.add(algo15[e][0])
            seeds_intersect5 = set(added_nodes5.intersection(seeds))
            seeds_intersect10 = set(added_nodes10.intersection(seeds))
            seeds_intersect15 = set(added_nodes15.intersection(seeds))
            df_lp5[f'alpha = {i}'][ele] += round(len(seeds_intersect5)/5 * 100, 2)
            df_lp10[f'alpha = {i}'][ele] += round(len(seeds_intersect10)/10 * 100, 2)
            df_lp15[f'alpha = {i}'][ele] += round(len(seeds_intersect15)/15 * 100, 2)
    
    # Subcellular location filtration
    for ele in x:
        for i in range(1, 6):
            added_nodes5 = set()
            added_nodes10 = set()
            added_nodes15 = set()
            algo5 = DIAMOnD(G_original = G_l, seed_genes = seeds1, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random5.csv')
            algo10 = DIAMOnD(G_original = G_l, seed_genes = seeds2, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random10.csv')
            algo15 = DIAMOnD(G_original = G_l, seed_genes = seeds3, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random15.csv')
            for e in range(ele):
                added_nodes5.add(algo5[e][0])
                added_nodes10.add(algo10[e][0])
                added_nodes15.add(algo15[e][0])
            seeds_intersect5 = set(added_nodes5.intersection(seeds))
            seeds_intersect10 = set(added_nodes10.intersection(seeds))
            seeds_intersect15 = set(added_nodes15.intersection(seeds))
            df_l5[f'alpha = {i}'][ele] += round(len(seeds_intersect5)/5 * 100, 2)
            df_l10[f'alpha = {i}'][ele] += round(len(seeds_intersect10)/10 * 100, 2)
            df_l15[f'alpha = {i}'][ele] += round(len(seeds_intersect15)/15 * 100, 2)
    
    # GO filtration
    for ele in x:
        for i in range(1, 6):
            added_nodes5 = set()
            added_nodes10 = set()
            added_nodes15 = set()
            algo5 = DIAMOnD(G_original = G_go, seed_genes = seeds1, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random5.csv')
            algo10 = DIAMOnD(G_original = G_go, seed_genes = seeds2, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random10.csv')
            algo15 = DIAMOnD(G_original = G_go, seed_genes = seeds3, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random15.csv')
            for e in range(ele):
                added_nodes5.add(algo5[e][0])
                added_nodes10.add(algo10[e][0])
                added_nodes15.add(algo15[e][0])
            seeds_intersect5 = set(added_nodes5.intersection(seeds))
            seeds_intersect10 = set(added_nodes10.intersection(seeds))
            seeds_intersect15 = set(added_nodes15.intersection(seeds))
            df_go5[f'alpha = {i}'][ele] += round(len(seeds_intersect5)/5 * 100, 2)
            df_go10[f'alpha = {i}'][ele] += round(len(seeds_intersect10)/10 * 100, 2)
            df_go15[f'alpha = {i}'][ele] += round(len(seeds_intersect15)/15 * 100, 2)
    
    # General reactome filtration + subcellular location filtration
    for ele in x:
        for i in range(1, 6):
            added_nodes5 = set()
            added_nodes10 = set()
            added_nodes15 = set()
            algo5 = DIAMOnD(G_original = G_pl, seed_genes = seeds1, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random5.csv')
            algo10 = DIAMOnD(G_original = G_pl, seed_genes = seeds2, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random10.csv')
            algo15 = DIAMOnD(G_original = G_pl, seed_genes = seeds3, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random15.csv')
            for e in range(ele):
                added_nodes5.add(algo5[e][0])
                added_nodes10.add(algo10[e][0])
                added_nodes15.add(algo15[e][0])
            seeds_intersect5 = set(added_nodes5.intersection(seeds))
            seeds_intersect10 = set(added_nodes10.intersection(seeds))
            seeds_intersect15 = set(added_nodes15.intersection(seeds))
            df_pl5[f'alpha = {i}'][ele] += round(len(seeds_intersect5)/5 * 100, 2)
            df_pl10[f'alpha = {i}'][ele] += round(len(seeds_intersect10)/10 * 100, 2)
            df_pl15[f'alpha = {i}'][ele] += round(len(seeds_intersect15)/15 * 100, 2)
    
    # GO, Reactome and Subcellular location filtering
    for ele in x:
        for i in range(1, 6):
            added_nodes5 = set()
            added_nodes10 = set()
            added_nodes15 = set()
            algo5 = DIAMOnD(G_original = G_gopl, seed_genes = seeds1, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random5.csv')
            algo10 = DIAMOnD(G_original = G_gopl, seed_genes = seeds2, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random10.csv')
            algo15 = DIAMOnD(G_original = G_gopl, seed_genes = seeds3, max_number_of_added_nodes = ele, alpha = i, outfile = 'BC_added_nodes_random15.csv')
            for e in range(ele):
                added_nodes5.add(algo5[e][0])
                added_nodes10.add(algo10[e][0])
                added_nodes15.add(algo15[e][0])
            seeds_intersect5 = set(added_nodes5.intersection(seeds))
            seeds_intersect10 = set(added_nodes10.intersection(seeds))
            seeds_intersect15 = set(added_nodes15.intersection(seeds))
            df_gopl5[f'alpha = {i}'][ele] += round(len(seeds_intersect5)/5 * 100, 2)
            df_gopl10[f'alpha = {i}'][ele] += round(len(seeds_intersect10)/10 * 100, 2)
            df_gopl15[f'alpha = {i}'][ele] += round(len(seeds_intersect15)/15 * 100, 2)

In [7]:
# Take the average of the 20 repititions
# No filtration
df_5 = df_5/10
df_5 = df_5.round(2)
df_10 = df_10/10
df_10 = df_10.round(2)
df_15 = df_15/10
df_15 = df_15.round(2)

# Reactome general filtration
df_p5 = df_p5/10
df_p5 = df_p5.round(2)
df_p10 = df_p10/10
df_p10 = df_p10.round(2)
df_p15 = df_p15/10
df_p15 = df_p15.round(2)

# Reactome leaf pathway filtration
df_lp5 = df_lp5/10
df_lp5 = df_lp5.round(2)
df_lp10 = df_lp10/10
df_lp10 = df_lp10.round(2)
df_lp15 = df_lp15/10
df_lp15 = df_lp15.round(2)

# Subcellular location filtration
df_l5 = df_l5/10
df_l5 = df_l5.round(2)
df_l10 = df_l10/10
df_l10 = df_l10.round(2)
df_l15 = df_l15/10
df_l15 = df_l15.round(2)

# GO filtration
df_go5 = df_go5/10
df_go5 = df_go5.round(2)
df_go10 = df_go10/10
df_go10 = df_go10.round(2)
df_go15 = df_go15/10
df_go15 = df_go15.round(2)

# General reactome filtration + subcellular location filtration
df_pl5 = df_pl5/10
df_pl5 = df_pl5.round(2)
df_pl10 = df_pl10/10
df_pl10 = df_pl10.round(2)
df_pl15 = df_pl15/10
df_pl15 = df_pl15.round(2)

# GO, Reactome and Subcellular location filtering
df_gopl5 = df_gopl5/10
df_gopl5 = df_gopl5.round(2)
df_gopl10 = df_gopl10/10
df_gopl10 = df_gopl10.round(2)
df_gopl15 = df_gopl15/10
df_gopl15 = df_gopl15.round(2)

# Save the dataframes

In [8]:
# No filtration
df_5.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_No_Filtration_5.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_10.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_No_Filtration_10.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_15.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_No_Filtration_15.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')

# Reactome general filtration
df_p5.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Reactome_general_5.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_p10.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Reactome_general_10.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_p15.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Reactome_general_15.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')

# Reactome leaf pathway filtration
df_lp5.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Reactome_leaf_5.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_lp10.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Reactome_leaf_10.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_lp15.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Reactome_leaf_15.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')

# Subcellular location filtration
df_l5.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Subcellular_location_5.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_l10.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Subcellular_location_10.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_l15.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Subcellular_location_15.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')

# GO filtration
df_go5.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_GO_5.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_go10.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_GO_10.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_go15.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_GO_15.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')

# General reactome filtration + subcellular location filtration
df_pl5.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Pathway_subcellular_location_5.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_pl10.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Pathway_subcellular_location_10.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_pl15.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_Pathway_subcellular_location_15.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')

# GO, Reactome and Subcellular location filtering
df_gopl5.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_GO_Pathway_subcellular_location_5.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_gopl10.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_GO_Pathway_subcellular_location_10.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')
df_gopl15.to_csv('D:\\Jana De Coster\\Documents\\Ugent\\2de master\\Master thesis\\Figures\\10_2_GO_Pathway_subcellular_location_15.csv', encoding = 'utf-8', compression = 'gzip', index = False, sep = '\t')